In [ ]:
# 一、选股,股池：
# (聚宽上我还没实现股池管理)
# 1、每日全市场选股入股池:
# (1)剔除st,退市、前一交易日停牌的股
# (2)选股条件：
# 连续三日缩量，当日量小于五日十日十五日均量
# (3)若标的已在股池中，选股日期设置为最新日期
# (4)每日清除在股池中超过5个交易日的标的
# 二、买入：
# 1、股池中符合买入条件:
# (1)当日量大于前一日量
# (2)当日量大于五日十日十五日均量
# (3)当日收盘价大于开盘价
# (4)当日KDJ金叉
# (5)买入前以换手及五日平均换手排序，换手大者优先买入(聚宽上我还没实现)
# (6)大盘23个交易日内出现macd死叉不买(刚想到，聚宽上我还没实现)


In [ ]:
import pandas as pd
import numpy as np

In [ ]:

def FILTERx(COND, N):

    k1 = pd.Series(np.where(COND, 1, 0), index=COND.index)
    idx = k1[k1 == 1].index.codes[0]
    needfilter = pd.Series(idx, index=idx)
    afterfilter = needfilter.diff().apply(lambda x: False if x > N else True)
    k1.iloc[afterfilter[afterfilter].index] = 2
    return k1.apply(lambda x: 1 if x == 2 else 0)

In [58]:
def select1(data):
    # 连续三日缩量
    ch= data.close * 1.1
    cl = data.close * 0.9
#     m5 = QA.MA(data.close, 5)
#     m10 = QA.MA(data.close, 10)
#     m20 = QA.MA(data.close, 20)
#     m30 = QA.MA(data.close, 30)
#     m60 = QA.MA(data.close, 60)
#     m90 = QA.MA(data.close, 90)
#     m120 = QA.MA(data.close, 120)
#     m250 = QA.MA(data.close, 250)
#     m500 = QA.MA(data.close, 5)
#     m750 = QA.MA(data.close, 5)
#     m900 = QA.MA(data.close, 5)
    
    MA_N = [QA.MA(data.close, x) for x in (5,10,20,30,60,90,120,250) ]
    t22 = pd.concat([QA.MA(data.close, x) for x in (5,10,20,30,60,90,120,250) ], axis = 1)
#     print(MA_N)
    
#     #当日量小于五日十日十五日均量
    t2 = pd.concat([QA.MA(data.volume.shift(1), x)-data.volume.shift(1) for x in [5,10,15]],axis=1).dropna().min(axis=1).apply(lambda x: True if x>0 else False)
#     t3 = pd.concat([t1,t2], sort=False, axis=1).all(axis=1)
#     res =t3.rolling(6).sum().apply(lambda x: True if x <6 and x>0 else False)
# #     #print(res)
    
    
#     x1 = data.volume.diff()>0
#     x2 = pd.concat([QA.MA(data.volume, x)-data.volume for x in [5,10,15]],axis=1).dropna().min(axis=1).apply(lambda x: True if x<0 else False)
#     x3 = data.close > data.open
    
#     kdj  = QA.QA_indicator_KDJ(data)
#     x4 = QA.CROSS(kdj.KDJ_K, kdj.KDJ_D).apply(lambda x: True if x==1 else False)
    
#     #xx = pd.concat([x1, x2, x3,x4], sort = False, axis=1 ).all(axis=1)
    
    
    return pd.DataFrame({
#         't1': t1,
#         't2': t2,
#         't3': t3,
        'm5': MA_N[0],
        'm10':  MA_N[1],
        'm20':  MA_N[2],
        'm30':  MA_N[3],
        'm60':  MA_N[4],
        'm90':  MA_N[5],
        'm120':  MA_N[6],
        'm250':  MA_N[7],
        't2':t2,
#         'man':MA_N
    })
    


In [ ]:
#QA.QA_fetch_stock_block_adv().get_block('上证50').code

In [ ]:
import QUANTAXIS as QA

In [ ]:
stock_list = QA.QA_fetch_stock_block_adv().get_block('中小300').code

In [ ]:
stock_list = QA.QA_fetch_stock_list_adv()

In [ ]:
stocklist_all = stock_list[~stock_list.name.apply(lambda x: 'ALL-ST' in x)].code.tolist()

In [ ]:
stocklist_all=stocklist_all[1:2]

In [ ]:
#blockname=QA.QA_fetch_stock_block_adv().block_name

In [ ]:
data = QA.QA_fetch_stock_day_adv(stocklist_all,'2000-01-01','2020-11-30')

In [ ]:
data.selects('000002','2018-06-10').data

In [ ]:
# = QA.QA_fetch_stock_day_adv('000001','2019-07-01','2019-09-30')

In [ ]:
#data.selects('600532','2008-05-28','2008-06-10').data

In [59]:
ind = data.add_func(select1)
ind

,,m5,m10,m20,m30,m60,m90,m120,m250,t2
date,code,,,,,,,,,
2000-01-04,000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-10,000002,10.656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-11,000002,10.764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-12,000002,10.848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-13,000002,10.834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-14,000002,10.642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ind

In [ ]:
x1 = ind.query('res==True').loc[:,['x1','x2','x3','x4']].all(axis=1).groupby(level=1).shift()


In [ ]:
x1[x1==True]

In [ ]:
ind.loc['2008-04-22':'2008-06-19']